In [38]:
import pandas as pd

In [39]:
path_for_work = r"C:\Users\User\Desktop"

In [40]:
name = r"\Sell_Out_Екатеринбург_Уралтабак" +"_SKU.xlsx"

In [41]:
osh = pd.read_excel(r"D:\Analysis Burn\Дистрибьютеры отчетность\Source\Отчеты\Екатеринбург Урал табак\Sell_Out_Екатеринбург_Уралтабак.xlsx")

In [42]:
osh.head(1)

,Торговый представитель,Код клиента,Сегмент,Клиент,Адрес доставки,Дата отгрузки,Бренд,Группа,Вкус,Граммовка,Количество шт,ВЕС,Цена,стоимость,Дистрибьютор
0,не указан,УТ0000697,Спец. магазин,Абдуллин Руслан Илхатович,"622016, Свердловская обл, Нижний Тагил г, Косм...",2023-03-01,Overdose,Overdose 25 гр,Melon Berry (Ягодная дыня),25,1,0.025,205.0,205.0,2


In [43]:
osh['Дата отгрузки'].dtypes == '<M8[ns]'

True

In [44]:
if osh['Дата отгрузки'].dtypes != '<M8[ns]':
    if osh['Дата отгрузки'].astype(str).str.match("\d{4}-\d{2}-\d{2}").all():
        osh['Дата отгрузки'] = pd.to_datetime(osh['Дата отгрузки'], yearfirst = True)
    elif osh['Дата отгрузки'].astype(str).str.match("\d{2}-\d{2}-\d{4}").all():
        osh['Дата отгрузки'] = pd.to_datetime(osh['Дата отгрузки'], dayfirst = True)
    else:
        show_notification("С датой что-то не так, проверьте плз")
        exit()

In [45]:
osh["Год"] = osh["Дата отгрузки"].dt.year
osh["Месяц"] = osh["Дата отгрузки"].dt.month

In [46]:
osh["Месяц"].unique()

array([3, 4, 5, 6], dtype=int64)

In [47]:
osh["Год"].unique()

array([2023], dtype=int64)

In [50]:
# пять месяцев отфильтровать
# osh = osh.loc[(osh['Год'] == 2023)]

In [51]:
osh_2 = osh.copy(deep=True)

In [52]:
osh_2.loc[:, 'Месяц название'] = osh_2.loc[:,'Год'].astype(str) + '-' + osh_2.loc[:, 'Месяц'].astype(str)



In [53]:
pivot_table = osh_2.pivot_table(values='ВЕС',
                             index=['Клиент', 'Бренд', 'Вкус', 'Группа'],
                             columns='Месяц название',
                             aggfunc=lambda x: 1 if sum(x) > 0 else 0,
                             fill_value=0).reset_index()

In [54]:
# сегмент только для ошишы
# 
# pivot_table = osh_2.pivot_table(values='ВЕС',
#                              index=['Сегмент','Клиент', 'Бренд', 'Вкус'],
#                              columns='Месяц название',
#                              aggfunc=lambda x: 1 if sum(x) > 0 else 0,
#                              fill_value=0).reset_index()

In [55]:
# здесь править цифру в iloc
pivot_table['Общая сумма'] = pivot_table.iloc[:, 4:].sum(axis=1)

In [56]:
# pivot_table.columns = pivot_table.columns.map(lambda x: '_'.join(map(str, x)))

In [57]:
pivot_table

Месяц название,Клиент,Бренд,Вкус,Группа,2023-3,2023-4,2023-5,2023-6,Общая сумма
0,АВРОРА ООО,BlackBurn,Ananas Shock (Кислый ананас),BlackBurn 25 гр,0,1,1,0,2
1,АВРОРА ООО,BlackBurn,Barberry Shock (Кислый барбарис),BlackBurn 25 гр,0,1,1,0,2
2,АВРОРА ООО,BlackBurn,ICE BABY (Ягодный сорбет с грейпфрутом) (COLLA...,BlackBurn 25 гр,0,1,0,1,2
3,АВРОРА ООО,BlackBurn,Melon Halls (Дынный холс),BlackBurn 25 гр,0,1,1,0,2
4,АВРОРА ООО,BlackBurn,Papaya v obed (Папайя) (COLLAB KMTM),BlackBurn 25 гр,0,1,1,0,2
...,...,...,...,...,...,...,...,...,...
17115,япона Пышма,BlackBurn,Something Berry (Что-то ягодное),BlackBurn 200 гр,0,1,0,0,1
17116,япона Пышма,BlackBurn,Watermelon (Сладкий арбуз),BlackBurn 200 гр,0,1,0,0,1
17117,япона Пышма,Blackburn,Haribon (Мармелад-кола),Blackburn 200 гр,0,0,0,1,1
17118,япона Пышма,Blackburn,Lemon Shock (Кислый лимон),Blackburn 200 гр,0,0,0,1,1


In [58]:
pivot_table.loc[pivot_table['Общая сумма'] == 0]

Месяц название,Клиент,Бренд,Вкус,Группа,2023-3,2023-4,2023-5,2023-6,Общая сумма
2005,Васин Иван Алексеевич,Overdose,"Brumblebee (Клубника, ежевика, голубика)",Overdose 200 гр,0,0,0,0,0
2009,Васин Иван Алексеевич,Overdose,Fantola (Тропическая газировка),Overdose 200 гр,0,0,0,0,0
2013,Васин Иван Алексеевич,Overdose,Frustyle (Кактус-лайм),Overdose 200 гр,0,0,0,0,0
2022,Васин Иван Алексеевич,Overdose,Manila Malina (Филиппинская малина),Overdose 200 гр,0,0,0,0,0
2035,Васин Иван Алексеевич,Overdose,Watermelon (Сахарный арбуз),Overdose 200 гр,0,0,0,0,0
2973,Гребенников Александр Леонидович,Overdose,Fantola (Тропическая газировка),Overdose 200 гр,0,0,0,0,0
2976,Гребенников Александр Леонидович,Overdose,Frustyle (Кактус-лайм),Overdose 200 гр,0,0,0,0,0
3689,Евдокимов Егор Владиславович,PeterRalf,Banana Сrepes (Утренний завтрак с бананом),PeterRalf 50 гр,0,0,0,0,0
4681,Замараева Светлана Евгеньевна,Blackburn,Lemon Shock (Кислый лимон),Blackburn 200 гр,0,0,0,0,0
6471,Козлова Анна Сергеевна,Blackburn,BlackBerry Lemonade (Ежевичный лимонад),Blackburn 200 гр,0,0,0,0,0


In [59]:
pivot_table['Общая сумма'].unique()

array([2, 3, 1, 4, 0], dtype=int64)

In [60]:
pivot_table3 = osh_2.pivot_table(values='ВЕС',
                             index=['Клиент', 'Бренд', 'Вкус'],
                             columns='Месяц название',
                             aggfunc=lambda x: 1 if sum(x) > 0 else 0,
                             fill_value=0).reset_index()

In [61]:
pivot_table3

Месяц название,Клиент,Бренд,Вкус,2023-3,2023-4,2023-5,2023-6
0,АВРОРА ООО,BlackBurn,Ananas Shock (Кислый ананас),0,1,1,0
1,АВРОРА ООО,BlackBurn,Barberry Shock (Кислый барбарис),0,1,1,0
2,АВРОРА ООО,BlackBurn,ICE BABY (Ягодный сорбет с грейпфрутом) (COLLA...,0,1,0,1
3,АВРОРА ООО,BlackBurn,Melon Halls (Дынный холс),0,1,1,0
4,АВРОРА ООО,BlackBurn,Papaya v obed (Папайя) (COLLAB KMTM),0,1,1,0
...,...,...,...,...,...,...,...
14787,япона Пышма,BlackBurn,Something Berry (Что-то ягодное),0,1,0,0
14788,япона Пышма,BlackBurn,Watermelon (Сладкий арбуз),0,1,0,0
14789,япона Пышма,Blackburn,Haribon (Мармелад-кола),0,0,0,1
14790,япона Пышма,Blackburn,Lemon Shock (Кислый лимон),0,0,0,1


In [62]:
# здесь править цифру в iloc
pivot_table3['Общая сумма'] = pivot_table3.iloc[:, 3:].sum(axis=1)

In [63]:
pivot_table2 = pivot_table3.pivot_table(values='Клиент',
                             index=['Бренд', 'Вкус'],
                             columns='Общая сумма',
                             aggfunc=pd.Series.count,
                             fill_value=0,
                             margins=False).reset_index()

In [64]:
# сегмент только ошише нужен
# pivot_table2 = pivot_table.pivot_table(values='Клиент',
#                              index=['Сегмент','Бренд', 'Вкус'],
#                              columns='Общая сумма',
#                              aggfunc=pd.Series.count,
#                              fill_value=0,
#                              margins=False).reset_index()

In [65]:
pivot_table2.head(2)

Общая сумма,Бренд,Вкус,0,1,2,3,4
0,B3,Orange Cream (Кремовый апельсин),0,1,0,0,0
1,Banger,BBQ (Соус Барбекю),0,10,5,0,0


In [1]:
columns = pivot_table.columns.values

pivot_table['Клиент_2'] = pd.NaT

# Добавление колонки 'Клиент1'
new_column = 'Клиент_2'
columns = [new_column] + [col for col in columns if col != new_column]

pivot_table = pivot_table[columns]

NameError: name 'pivot_table' is not defined

In [83]:
with pd.ExcelWriter(path_for_work + name) as writer:  
    pivot_table2.to_excel(writer, sheet_name='1', index=False)  
    pivot_table.to_excel(writer, sheet_name='2', index=False)